In [1]:
# imports and definitions

import libsbml
import importlib
import amici
import os
import sys
import numpy as np
import pandas as pd
import pypesto

dir_name = '/home/yannik/pypesto/doc/example/Fujita2010/'
model_name = 'Fujita_SciSignal2010'
model_output_dir = model_name

rel_sbml_file = 'SBML/model1_data1_l2v4.xml'
sbml_file = dir_name + rel_sbml_file

rel_info_file = 'General_info.xlsx'
info_file = dir_name + rel_info_file

rel_data_file = 'Data/model1_data1.xlsx'
data_file = dir_name + rel_data_file

In [2]:
# read in excel files

sim_data = pd.read_excel(data_file, sheet_name='Simulation')
obs_names = list(sim_data)[1:]
timepoints = np.array(sim_data['time'])

exp_data = pd.read_excel(data_file, sheet_name='Exp Data')

info = pd.read_excel(info_file, sheet_name='Experimental conditions', 
                     header=2)

In [34]:
# compile model

sbml_importer = amici.SbmlImporter(sbml_file)
_obs = amici.assignmentRules2observables(
    sbml_importer.sbml,
    filter_function=lambda var_id: var_id in obs_names)
fixedParameters=['EGF']
sbml_importer.sbml2amici(model_name, model_output_dir,
                         verbose=False,
                         observables=_obs,
                         constantParameters=fixedParameters)

libSBML Warning (SBML unit consistency): In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. 
 The units of the <initialAssignment> <math> expression 'init_EGFR' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.

libSBML Warning (SBML unit consistency): In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. 
 The units of the <initialAssignment> <math> expression 'init_Akt' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.

libSBML Warning (SBML unit consistency): In situations where a mathematical expressio

In [3]:
# load model

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()

model.requireSensitivitiesForAllParameters()
model.setParameterScale(2)
model.setParameters(amici.DoubleVector(np.log10(np.array(model.getParameters()))))
model.setFixedParameters(amici.DoubleVector(np.log10(np.array([0.1]))))
model.setTimepoints(amici.DoubleVector(np.array(timepoints, dtype=float)))

#print(list(model.getFixedParameters()))
#print(list(model.getFixedParameterIds()))
#print(list(model.getParameters()))
#print(list(model.getParameterIds()))
#print(list(model.getObservableIds()))
#print(list(model.getTimepoints()))

solver = model.getSolver()
solver.setSensitivityMethod(1)
solver.setSensitivityOrder(1)
solver.setAbsoluteTolerance(1e-10)
solver.setRelativeTolerance(1e-6)
solver.setMaxSteps(int(1e6))

sim = amici.runAmiciSimulation(model, solver)
print(sim) # works (compare with excel file)

{'ptr': <amici.amici.ReturnDataPtr; proxy of <Swig Object of type 'std::unique_ptr< amici::ReturnData > *' at 0x7f28f0934a50> >, 't': array([   0.,   60.,  120.,  300.,  600.,  900., 1800., 3600.]), 'x': array([[3.75361538e+07, 0.00000000e+00, 0.00000000e+00, 2.69309230e-02,
        0.00000000e+00, 1.88063149e+02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [3.36320136e+07, 2.20975380e+05, 8.81854694e-05, 2.59794662e-02,
        8.53400517e-04, 1.88063139e+02, 9.87080896e-06, 7.42463901e-10,
        3.53003786e+05],
       [3.01552361e+07, 3.31232092e+05, 1.23377210e-04, 2.42482010e-02,
        2.48756537e-03, 1.88063077e+02, 7.17794946e-05, 1.22174148e-08,
        3.72034167e+05],
       [2.17485676e+07, 2.77996776e+05, 9.72387343e-05, 2.27707065e-02,
        3.62842741e-03, 1.88062714e+02, 4.34550364e-04, 2.37658894e-07,
        2.76772657e+05],
       [1.26237045e+07, 1.61610530e+05, 5.85458821e-05, 2.35832506e-02,
        2.38201997e-03, 1.88062240e+02, 9.071065

In [ ]:
# get edata

edata = amici.ExpData(sim['ptr'].get(), 0, 0)
edata.setTimepoints(amici.DoubleVector(np.array(timepoints, dtype=float)))
edata.setObservedData(amici.DoubleVector(exp_data['pEGFR_tot']), 0)
print(list(edata.getObservedData()))